In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.
/home/Jagan/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/Jagan/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Get images
image = img_to_array(load_img('woman.jpg'))
image = np.array(image, dtype=float)

In [3]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128 # why divide by 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [4]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [5]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [6]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=100)

Epoch 1/100
1/1 [==============================] - 23s 23s/step - loss: 0.1115
Epoch 2/100
1/1 [==============================] - 0s 23ms/step - loss: 0.8037
Epoch 3/100
1/1 [==============================] - 0s 21ms/step - loss: 0.0357
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 0.0109
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0077
Epoch 6/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0066
Epoch 7/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0062
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0066
Epoch 9/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0071
Epoch 10/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0061
Epoch 11/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0056
Epoch 12/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0053
Epoch 13/100
1/1 [==============================]

In [7]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

1/1 [==============================] - 0s 69ms/step
0.0027718401979655027


/home/Jagan/anaconda3/envs/fastai/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/home/Jagan/anaconda3/envs/fastai/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
